In [90]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances

In [91]:
house_data = pd.read_json('/Users/admin/Desktop/Two sigma/train.json')
house_data.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street


In [92]:
house_data.shape

(49352, 15)

In [93]:
house_data.isnull().sum()

bathrooms          0
bedrooms           0
building_id        0
created            0
description        0
display_address    0
features           0
interest_level     0
latitude           0
listing_id         0
longitude          0
manager_id         0
photos             0
price              0
street_address     0
dtype: int64

In [94]:
feature_data = house_data[['description','features','interest_level']]
feature_data.head()

,description,features,interest_level
10,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,[],medium
10000,,"[Doorman, Elevator, Fitness Center, Cats Allow...",low
100004,"Top Top West Village location, beautiful Pre-w...","[Laundry In Building, Dishwasher, Hardwood Flo...",high
100007,Building Amenities - Garage - Garden - fitness...,"[Hardwood Floors, No Fee]",low
100013,Beautifully renovated 3 bedroom flex 4 bedroom...,[Pre-War],low


In [95]:
feature_value = feature_data['features'].tolist()
feature_value

feature_lst = []
for i in range(len(feature_value)):
    feature_lst += feature_value[i]
# print(len(feature_lst)) # all features

uniq_feature = list(set(feature_lst))
# print(uniq_feature) #all unique features
len(uniq_feature)


1556

In [96]:
# see the frequency of each feature
import collections
def most_common(lst):
    features = collections.Counter(lst)
    feature_value = features.keys()
    frequency = features.values()
    data = [('feature_value', feature_value),
            ('frequency', frequency),]    
    df = pd.DataFrame.from_items(data)
    return df.sort_values(by = 'frequency', ascending = False)

most_common(feature_lst)


,feature_value,frequency
639,Elevator,25915
974,Cats Allowed,23540
1018,Hardwood Floors,23527
1203,Dogs Allowed,22035
22,Doorman,20898
1316,Dishwasher,20426
826,No Fee,18062
168,Laundry in Building,16344
880,Fitness Center,13252
288,Pre-War,9148


In [100]:
def newColumn(name,df,series):
    feature = pd.Series(0,df.index,name = name)# data : 0
    for row,word in enumerate(series):
        if name in word:
            feature.iloc[row] = 1
    df[name] = feature # feature : series ; value in series : 1 or 0
    return df

# select features based on frequency
facilities = ['Elevator','Cats Allowed','Hardwood Floors','Dogs Allowed','Doorman','Dishwasher','No Fee','Laundry in Building','Fitness Center']
for name in facilities:
    house_data = newColumn(name, house_data, house_data['features'])
print(house_data)


        bathrooms  bedrooms                       building_id  \
10            1.5         3  53a5b119ba8f7b61d4e010512e0dfc85   
10000         1.0         2  c5c8a357cba207596b04d1afd1e4f130   
100004        1.0         1  c3ba40552e2120b0acfc3cb5730bb2aa   
100007        1.0         1  28d9ad350afeaab8027513a3e52ac8d5   
100013        1.0         4                                 0   
100014        2.0         4  38a913e46c94a7f46ddf19b756a9640c   
100016        1.0         2  3ba49a93260ca5df92fde024cb4ca61f   
100020        2.0         1  0372927bcb6a0949613ef5bf893bbac7   
100026        1.0         1  a7efbeb58190aa267b4a9121cd0c88c0   
100027        2.0         4                                 0   
100030        1.0         0                                 0   
10004         1.0         1                                 0   
100044        1.0         2  67c9b420da4a365bc26a6cd0ef4a5320   
100048        2.0         2                                 0   
10005         1.0        